In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import ticker
from matplotlib import colors
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import math
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Functions

In [8]:
def plot_streamlines(case, dirname, fname, dep_ind):

    arrow = 1
    days = 9

    vozocrtx, vomecrty, umask, vmask, e1u, e2u, depthu, depthv, sozotaux = quicklook.get_uv_at_depth(dirname, fname, dep_ind)

    x_slice, y_slice, u_nstg, v_nstg, speeds = quicklook.get_speeds(vozocrtx, vomecrty, arrow)
    speeds_daily = quicklook.get_1day_avg_of_speeds(speeds, days)
    u_nstg_daily = quicklook.get_1day_avg_of_speeds(u_nstg, days)
    v_nstg_daily = quicklook.get_1day_avg_of_speeds(v_nstg, days)

    # ----------------------------------------------------------------------------------------------------

    umask_slice = umask[:-1, :-1]
    vmask_slice = vmask[:-1, :-1]

    vmin, vmax = 0, 0.7
    cmap = cmo.cm.matter
    cmap.set_bad('silver')
    cmap_mask = LinearSegmentedColormap.from_list('mycmap', ['silver', 'white'])
    norm = mpl.colors.Normalize(vmin=vmin,vmax=vmax)

    fig, axes = plt.subplots(3, 3, figsize = (20, 21))
    for ax, n in zip(axes.flatten(), np.arange(9)):
        plot_speeds = np.ma.array(speeds_daily[n, ...], mask=1 - umask_slice)
        plot_u_nstg = np.ma.array(u_nstg_daily[n, ...], mask=1 - umask_slice)
        plot_v_nstg = np.ma.array(v_nstg_daily[n, ...], mask=1 - vmask_slice)
        strm = ax.streamplot(x_slice, y_slice, plot_u_nstg, plot_v_nstg, color=plot_speeds,
                             cmap=cmap, norm=norm, linewidth=2, density=6, zorder=2) #density different
        ax.pcolormesh(umask_slice, cmap=cmap_mask, zorder=1)
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_aspect(aspect='equal')
        ax.set_xlim([0, x_slice[-1]])
        ax.set_ylim([0, y_slice[-1]])
        ax.set_title('Day ' + str(n+1), fontsize=20)
        cbar = fig.colorbar(strm.lines, ax=ax, fraction=0.05, orientation='horizontal', pad=0.009)
        cbar.ax.tick_params(labelsize=13)
        tick_locator = ticker.MaxNLocator(nbins=5)
        cbar.locator = tick_locator
        cbar.update_ticks()

    fig.tight_layout(w_pad=1.2, h_pad=0.01)
    fig.suptitle(case+' - Streamlines with speeds [m/s] at depth = '+str(int(depthu[dep_ind]))+' m', fontsize=25)
    plt.subplots_adjust(top=0.96)
    
    return fig

# Constants

In [9]:
dep_inds = [1, 9, 16]

In [10]:
fname = "1_MCKNZ_1h_20170101_201701*"

# Ideal

In [11]:
kind = 'ideal'
case = 'base' 

for dep_ind, n in zip(dep_inds, np.arange(3)):
    dirname = '/ocean/imachuca/Canyons/results_mackenzie/extended_domain/'+kind+'_'+case+'/'
    fig = plot_streamlines(case, dirname, fname, dep_ind)
    fig.savefig('../writing_images/streamlines_'+kind+'_'+case+'_'+str(dep_ind)+'.png', dpi=100, bbox_inches='tight')
    plt.close(fig) 